In [9]:
import numpy as np
import matplotlib.pyplot as plt

# Misc
import warnings
import tensorflow as tf
import keras 
warnings.filterwarnings('ignore')

print(sys.version)
print(os.path)
print("Keras: ",keras.__version__)
print("TF: ",tf.__version__)
print("\n\nYou are good to go")

#
from approximator import *
from q_learning_nn import *
from evaluation import *

#import environment
sys.path.append(r'../virl')
import virl

#time
import time
from datetime import date


3.8.5 (default, Sep  4 2020, 02:22:02) 
[Clang 10.0.0 ]
<module 'posixpath' from '/Users/wayenvan/opt/anaconda3/lib/python3.8/posixpath.py'>
Keras:  2.4.3
TF:  2.3.1


You are good to go


## Training phase
define some properities

In [4]:
#create training environment, I am using 
alpha = 0.004
nn_config = [36,36]
BATCH_SIZE = 80
BUFFER_SIZE = 8000
num_episodes = 1000

start training

In [5]:
for num_instance in range(7, 10):
  env_train = virl.Epidemic(problem_id=num_instance, stochastic=False, noisy=False) 
  d_states = env_train.observation_space.shape[0]
  n_actions = env_train.action_space.n

  # Init the tow networks
  nn_func_approximator = NNFunctionApproximatorJointKeras(alpha, d_states, n_actions, nn_config)
  nn_func_approximator_target = NNFunctionApproximatorJointKeras(alpha, d_states, n_actions, nn_config)

  # Training   
  stats = q_learning_nn(env_train, nn_func_approximator, nn_func_approximator_target, num_episodes,
                  use_normalization = True,
                  max_steps_per_episode=500,discount_factor=0.9, 
                  epsilon_init=0.1,epsilon_decay=0.99995,epsilon_min=0.001,
                  use_batch_updates=True, BATCH_SIZE=BATCH_SIZE,
                  fn_model_in=None, fn_model_out=r"./data/approximator_training/cartpole0"+str(num_instance)+".h5", BUFFER_SIZE=BUFFER_SIZE)
  
  #save stats
  stats_storage = Stats_storage(stats)
  save_variable(stats_storage, "./data/stats_training/stats_train0"+str(num_instance)+".dat")

  #output log
  t = time.localtime()
  current_time = time.strftime("%H:%M:%S", t)

  #output log
  date = date.today()
  log = open("./data/training.log", "a")
  log.write("problem_id = {} finished training in {} {}\n".format(num_instance, current_time, date))
  log.close()

episode: 999/1000, score: 51, epsilon: 0.0075

## test every episode

In [10]:
#create training environment, I am using 
alpha = 0.00  #only run episode, not evaluation 
nn_config = [36,36]
BATCH_SIZE = 80
BUFFER_SIZE = 8000
num_episodes = 200

In [11]:

#load test environment

for num_instance in range(1):
    
    #run different instance
    for s_n in range(4):

        #set stochastic and noisy from binary(00) to binary(11)
        stochastic = bool((s_n//2)%2)
        noisy = bool(s_n%2) 

        env = virl.Epidemic(problem_id=num_instance, stochastic=stochastic, noisy=noisy)

        d_states = env.observation_space.shape[0]
        n_actions = env.action_space.n

        nn_approximator =  NNFunctionApproximatorJointKeras(alpha, d_states, n_actions, nn_config)
        nn_approximator_target =  NNFunctionApproximatorJointKeras(alpha, d_states, n_actions, nn_config)

        # Training   
        stats = q_learning_nn(env, nn_approximator, nn_approximator_target, num_episodes,
                        use_normalization = True,
                        max_steps_per_episode=500, 
                        epsilon_init=0.0, epsilon_decay=1.0, epsilon_min=0.00,
                        use_batch_updates=True, BATCH_SIZE=BATCH_SIZE,
                        fn_model_in=r"./data/approximator_training/cartpole0"+str(num_instance)+".h5", fn_model_out=None,
                        BUFFER_SIZE=BUFFER_SIZE) 
        #save stats
        stats_storage = Stats_storage(stats)
        save_variable(stats_storage, "./data/stats_evaluation/stats_eval0{}_{}{}.dat".format(num_instance, int(stochastic), int(noisy)))

         #output log
        t = time.localtime()
        current_time = time.strftime("%H:%M:%S", t)
        date = date.today()
        log = open("./data/evaluation.log", "a")
        log.write("problem_id = {} with stochatstic={} and noisy={} finished testing in {} {}\n".format(num_instance, stochastic, noisy, current_time, date))
        log.close()

episode: 128/200, score: 51, epsilon: 0.0

KeyboardInterrupt: 